# July 11th Updates: Matching and Filtering H-1B and Revelio Data

In [ ]:
### PACKAGE/MODULE IMPORTS AND PATHS
import duckdb as ddb
import sys 

sys.path.append('../')
from config import * 

# # helper functions
# sys.path.append('02_revelio_indiv_clean/')
# import rev_indiv_clean_helpers as help

con = ddb.connect()

# Importing Country Codes Crosswalk
with open(f"{root}/data/crosswalks/country_dict.json", "r") as json_file:
    country_cw_dict = json.load(json_file)

# helper function to get standardized country name
def get_std_country(country, dict = country_cw_dict):
    if country is None:
        return None 
    
    if country in dict.keys():
        return dict[country]
    
    if country in dict.values():
        return country 
    
    return "No Country Match"

con.create_function("get_std_country", lambda x: get_std_country(x), ['VARCHAR'], 'VARCHAR')


In [ ]:
### RAW DATA IMPORT
## duplicate rcids (companies that appear more than once in linkedin data)
dup_rcids = con.read_csv(f"{root}/data/int/dup_rcids_mar20.csv")

## matched company data from R
rmerge = con.read_csv(f"{root}/data/int/good_match_ids_mar20.csv")

## raw FOIA bloomberg data
foia_raw_file = con.read_csv(f"{root}/data/raw/foia_bloomberg/foia_bloomberg_all.csv")

## joining raw FOIA data with merged data to get foia_ids in raw foia data
foia_with_ids = con.sql("SELECT *, CASE WHEN matched IS NULL THEN 0 ELSE matched END AS matchind FROM ((SELECT * FROM foia_raw_file WHERE NOT FEIN = '(b)(3) (b)(6) (b)(7)(c)') AS a LEFT JOIN (SELECT lottery_year, FEIN, foia_id, 1 AS matched FROM rmerge GROUP BY lottery_year, FEIN, foia_id) AS b ON a.lottery_year = b.lottery_year AND a.FEIN = b.FEIN)")

# Importing User x Education-level Data (From WRDS Server)
rev_raw = con.read_parquet(f"{wrds_out}/rev_user_merge0.parquet")

for j in range(1,10):
    rev_raw = con.sql(f"SELECT * FROM rev_raw UNION ALL SELECT * FROM '{wrds_out}/rev_user_merge{j}.parquet'")
    print(rev_raw.shape)

# Importing Institution x Country Matches
inst_country_cw = con.read_parquet(f"{root}/data/int/rev_inst_countries_jun30.parquet")

# Importing Name x Country Matches
nanats = con.read_parquet(f"{root}/data/int/name2nat_revelio/rev_names_withnat_jun26.parquet")
nts = con.read_parquet(f'{root}/data/int/rev_names_nametrace_jul8.parquet')

# Importing User x Position-level Data
merged_pos = con.read_parquet(f"{root}/data/int/rev_merge_mar20.parquet")
#occ_cw = con.read_csv(f"{root}/data/crosswalks/rev_occ_to_foia_freq.csv")

### CLEANED DATA
## company crosswalk
company_cw = con.read_parquet(f"{root}/data/int/company_merge_sample_jun30.parquet")

## revelio individual user x country cleaned data
rev_users = con.read_parquet(f'{root}/data/int/rev_users_clean_jun30.parquet')

## revelio user x position history data
rev_positionhist = con.read_parquet(f'{root}/data/wrds/wrds_out/rev_user_positionhist_jul2.parquet')


(16878855, 19)
(23850622, 19)
(32457059, 19)
(39418941, 19)
(47525355, 19)
(55425986, 19)
(62847741, 19)
(71625007, 19)
(79574137, 19)


IOException: IO Error: No files found that match the pattern "/Users/amykim/Princeton Dropbox/Amy Kim/h1bworkers/data/int/rev_names_nametrace_jul8.parquet"

## 'Left-Hand Side': H-1B Applications

In [3]:
# RAW H-1B APPLICATIONS

# 1. total
# 2. multiple reg
# 3. small emp (final samp)

# 4. further cuts: suff diversity, 5. repeated lottery tries

## 'Right-Hand Side': LinkedIn Profiles

In [ ]:
# 